In [ ]:
import pandas as pd
import spacy
from tqdm import tqdm_notebook as tqdm
import numpy as np 
import pandas as pd 
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
import re
from keras.utils import to_categorical
import random
# from tensorflow import set_random_seed
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense,Dropout,Embedding,LSTM
from keras.callbacks import EarlyStopping
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.models import Sequential
import warnings
import json
import gc
%matplotlib inline
lemmatizer = WordNetLemmatizer()


# New Section

## Data Preprocessing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%time
dtypes = {'id':'object', 'flag':'category', 'user':'object', 'text':'object'}
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/twitter_train_set.csv',dtype=dtypes)
df.loc[df.label == 4, 'label'] = 1

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.87 µs


In [ ]:
df.head()

,label,id,date,flag,user,text
0,1,1973775248,Sat May 30 11:48:37 PDT 2009,NO_QUERY,mjs4320,Home from work!
1,0,1974556938,Sat May 30 13:22:31 PDT 2009,NO_QUERY,dougc84,@maeband did you guy's show sell out? i can't...
2,0,2202399576,Tue Jun 16 21:43:52 PDT 2009,NO_QUERY,Rebekkahcolburn,Did my poetry reading at kevro's tonight even ...
3,1,1962899636,Fri May 29 11:53:23 PDT 2009,NO_QUERY,addyct_d,google wave sounds really amazing... new way o...
4,0,1469846621,Tue Apr 07 07:35:31 PDT 2009,NO_QUERY,dilharasays,@thegutterflower sigh. true that.


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1440001 entries, 0 to 1440000
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype   
---  ------  --------------    -----   
 0   label   1440001 non-null  int64   
 1   id      1440001 non-null  object  
 2   date    1440001 non-null  object  
 3   flag    1440001 non-null  category
 4   user    1440001 non-null  object  
 5   text    1440001 non-null  object  
dtypes: category(1), int64(1), object(4)
memory usage: 56.3+ MB


In [ ]:
# Sample for the texe
sample = df.text[12]
sample

"@MorganBrianna I got a dodge intrepid 99' and thankxx @jbuckk I'm gladd yu approve "

In [ ]:
nlp = spacy.load("en_core_web_sm")
doc = nlp(sample)
len(doc)

16

In [ ]:
text_list = []
for sent in tqdm(df['text']):
    review_text = BeautifulSoup(sent).get_text()
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    text_list.append(review_text)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [ ]:
with open('phase_clean.json', 'w', encoding='utf-8') as file:
    json.dump(text_list, file)

In [ ]:
with open('phase_clean.txt', 'w') as file:
    for ele in tqdm(text_list):
        file.write(ele.strip() + "\n")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
words_list = []
for text in tqdm(text_list):
    #tokenize the sentences
    words = word_tokenize(text.lower())
    
    #lemmatize each word to its lemma
    
    lemma_words = [lemmatizer.lemmatize(i) for i in words]
    
    words_list.append(lemma_words)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [ ]:
with open('phase_clean.json', 'r') as file:
  text_list = json.load(file)
with open('word_list.json', 'w') as file:
    json.dump(words_list, file)
    

In [ ]:
with open('word_list.json', 'r') as file:
    words_list = json.load(file)

In [ ]:
words_list[0]

['home', 'from', 'work']

In [ ]:
# lemmatizer = WordNetLemmatizer()
# def clean_sentences(df):
#     reviews = []
#     words_list = []

#     for sent in tqdm(df['text']):
        
#         #remove html content
#         review_text = BeautifulSoup(sent).get_text()
        
#         #remove non-alphabetic characters
#         review_text = re.sub("[^a-zA-Z]"," ", review_text)
    
#         #tokenize the sentences
#         words = word_tokenize(review_text.lower())
    
#         #lemmatize each word to its lemma
#         lemma_words = [pattern.en.lemma(i) for i in words]
    
#         reviews.append(lemma_words)

#     return(reviews)

In [ ]:
# Get unique word
unique_words = set()
len_max = 0
for sent in tqdm(words_list):
    unique_words.update(sent)
    if(len_max<len(sent)):
        len_max = len(sent)
        
#length of the list of unique_words gives the no of unique words
print(len(list(unique_words)))
print(len_max)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.



528469
52


In [ ]:
y_target = to_categorical(df['label'], num_classes = 2)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(words_list,y_target,test_size=0.2,stratify=df['label'])

In [ ]:
num_classes = 2

In [ ]:
tokenizer = Tokenizer(num_words=len(list(unique_words)))
tokenizer.fit_on_texts(list(words_list))
X_train = tokenizer.texts_to_sequences(X_train)
X_val = tokenizer.texts_to_sequences(X_val)

#padding done to equalize the lengths of all input reviews. LSTM networks needs all inputs to be same length.
#Therefore reviews lesser than max length will be made equal using extra zeros at end. This is padding.
X_train = sequence.pad_sequences(X_train, maxlen=len_max)
X_val = sequence.pad_sequences(X_val, maxlen=len_max)
print(X_train.shape,X_val.shape)

(1152000, 52) (288001, 52)


In [ ]:
tokenizer_json = tokenizer.to_json()
with open('tokenizer.json', 'w', encoding='utf-8') as f:
    json.dump(tokenizer_json,f, ensure_ascii=False)

In [ ]:
early_stopping = EarlyStopping(min_delta = 0.001, mode = 'max', monitor='val_acc', patience = 2)
callback = [early_stopping]

#Model using Keras LSTM
model=Sequential()
model.add(Embedding(len(list(unique_words)),300,input_length=len_max))
model.add(LSTM(128,dropout=0.5, recurrent_dropout=0.5,return_sequences=False))
# model.add(LSTM(64,dropout=0.5, recurrent_dropout=0.5,return_sequences=False))
model.add(Dense(100,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.005),metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 52, 300)           158540700 
_________________________________________________________________
lstm (LSTM)                  (None, 128)               219648    
_________________________________________________________________
dense (Dense)                (None, 100)               12900     
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 202       
Total params: 158,773,450
Trainable params: 158,773,450
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history=model.fit(X_train, y_train, validation_data=(X_val, y_val),
                  epochs=6, 
                  batch_size=256, 
                  verbose=1, 
                  callbacks=callback)

In [ ]:
model.save('sentiment1.h5')

## Model evaluation